In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import random
import datetime
import os

In [59]:
data=pd.read_csv("/kaggle/input/solar-thing/accurate_solar_power_dataset.csv")
data.head()
data=data.drop(columns=['weather_condition','is_raining'])

In [60]:
data.isnull().sum()

datetime                   0
hour                       0
day_of_year                0
month                      0
latitude                   0
longitude                  0
panel_surface_area_m2      0
panel_tilt_degrees         0
panel_azimuth_degrees      0
panel_efficiency           0
panel_technology           0
solar_elevation_degrees    0
solar_irradiance_w_m2      0
temperature_celsius        0
humidity_percent           0
wind_speed_ms              0
cloud_cover_percent        0
seasonal_factor            0
power_output_kw            0
dtype: int64

In [61]:
X=data[[	'hour','day_of_year',	'month', 'latitude',	'longitude',	'panel_surface_area_m2',	'panel_tilt_degrees',	'panel_azimuth_degrees',	'panel_efficiency', 'solar_irradiance_w_m2',	'solar_elevation_degrees',	'temperature_celsius',	'humidity_percent',	'wind_speed_ms',	'cloud_cover_percent',	'seasonal_factor']]
y=data['power_output_kw']

In [62]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [64]:
y_pred = rf.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

MAE: 0.42366403500000016
R²: 0.9935954232136328


In [65]:
def predict_power(hour, day_of_year, month, latitude, longitude,
                  panel_surface_area_m2, panel_tilt_degrees, panel_azimuth_degrees,
                  panel_efficiency, solar_irradiance_w_m2, solar_elevation_degrees,
                  temperature_celsius, humidity_percent, wind_speed_ms,
                  cloud_cover_percent, seasonal_factor):
    
    features = [[hour, day_of_year, month, latitude, longitude,
                 panel_surface_area_m2, panel_tilt_degrees, panel_azimuth_degrees,
                 panel_efficiency, solar_irradiance_w_m2, solar_elevation_degrees,
                 temperature_celsius, humidity_percent, wind_speed_ms,
                 cloud_cover_percent, seasonal_factor]]
    
    return rf.predict(features)[0]

In [1]:
import datetime
import random

city_to_coords = {
    "delhi": (28.6, 77.2),
    "mumbai": (19.0, 72.8),
    "bengaluru": (12.9, 77.6),
    "bangalore": (12.9, 77.6),  # Added common alias
    "chennai": (13.1, 80.3),
    "kolkata": (22.6, 88.4),
    "ahmedabad": (23.0, 72.6)  # Added Ahmedabad coordinates
}

def prepare_inputs(city, panel_surface_area_m2, panel_tilt_degrees,
                   panel_azimuth_degrees, panel_efficiency):
    # --- get datetime features
    now = datetime.datetime.now()
    hour = now.hour
    day_of_year = now.timetuple().tm_yday
    month = now.month
    
    # --- get lat/lon from city (with error handling)
    city_lower = city.lower().strip()
    if city_lower not in city_to_coords:
        available_cities = ", ".join(city_to_coords.keys())
        raise ValueError(f"City '{city}' not found. Available cities: {available_cities}")
    
    latitude, longitude = city_to_coords[city_lower]
    
    # --- mock values for other features
    mock_inputs = {
        "hour": hour,
        "day_of_year": day_of_year,
        "month": month,
        "latitude": latitude,
        "longitude": longitude,
        "panel_surface_area_m2": panel_surface_area_m2,
        "panel_tilt_degrees": panel_tilt_degrees,
        "panel_azimuth_degrees": panel_azimuth_degrees,
        "panel_efficiency": panel_efficiency,
        "solar_irradiance_w_m2": random.uniform(600, 1000),
        "solar_elevation_degrees": random.uniform(30, 70),
        "temperature_celsius": random.uniform(20, 40),
        "humidity_percent": random.uniform(30, 70),
        "wind_speed_ms": random.uniform(0, 5),
        "cloud_cover_percent": random.uniform(0, 50),
        "seasonal_factor": 1.0
    }
    return mock_inputs

# Mock prediction function (replace with your actual model)
def predict_power(**kwargs):
    # Simple mock calculation - replace with your actual trained model
    base_power = (kwargs['panel_surface_area_m2'] * kwargs['panel_efficiency'] * 
                  kwargs['solar_irradiance_w_m2'] / 1000)
    return round(base_power, 2)

# For Kaggle - use sample values instead of input()

# Sample inputs - modify these values as needed
city = "Bangalore"  # Change this to test different cities
panel_area = 2.0  # m²
tilt = 37.8  # degrees
azimuth = 180.0  # degrees (south-facing)
efficiency = 0.50  # 20% efficiency

print(f"Testing solar panel configuration:")
print(f"City: {city}")
print(f"Panel Area: {panel_area} m²")
print(f"Tilt: {tilt}°")
print(f"Azimuth: {azimuth}°")
print(f"Efficiency: {efficiency}")
print("-" * 40)

try:
    # Build all required features
    inputs = prepare_inputs(city, panel_area, tilt, azimuth, efficiency)
    
    # Make prediction
    predicted = predict_power(**inputs)
    print(f"Predicted Power Output: {predicted} kW")
    
    # Print some of the input features for verification
    print("\nInput features used:")
    for key, value in inputs.items():
        print(f"{key}: {value}")
        
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Testing solar panel configuration:
City: Bangalore
Panel Area: 2.0 m²
Tilt: 37.8°
Azimuth: 180.0°
Efficiency: 0.5
----------------------------------------
Predicted Power Output: 0.94 kW

Input features used:
hour: 20
day_of_year: 256
month: 9
latitude: 12.9
longitude: 77.6
panel_surface_area_m2: 2.0
panel_tilt_degrees: 37.8
panel_azimuth_degrees: 180.0
panel_efficiency: 0.5
solar_irradiance_w_m2: 935.7779840355995
solar_elevation_degrees: 68.1509723491078
temperature_celsius: 29.05607123597296
humidity_percent: 41.01091677548096
wind_speed_ms: 3.150199817219845
cloud_cover_percent: 47.69621019375613
seasonal_factor: 1.0


In [67]:
# set seed for reproducibility
np.random.seed(42)

# define ranges
AREA_RANGE = (1.0, 10.0)   # m²
EFF_RANGE  = (0.12, 0.22)  # efficiency fraction
TILT_RANGE = (0.0, 45.0)   # degrees
AZ_RANGE   = (90.0, 270.0) # degrees

# create synthetic panel features
data['panel_surface_area_m2'] = np.random.uniform(*AREA_RANGE, size=len(data))
data['panel_efficiency']     = np.random.uniform(*EFF_RANGE, size=len(data))
data['panel_tilt_degrees']   = np.random.uniform(*TILT_RANGE, size=len(data))
data['panel_azimuth_degrees'] = np.random.uniform(*AZ_RANGE, size=len(data))

# quick check
data[['panel_surface_area_m2','panel_efficiency','panel_tilt_degrees','panel_azimuth_degrees']].head()


,panel_surface_area_m2,panel_efficiency,panel_tilt_degrees,panel_azimuth_degrees
0,4.370861,0.157364,32.849924,204.866022
1,9.556429,0.153291,8.303040,172.672642
2,7.587945,0.137615,15.598786,263.609734
3,6.387926,0.180727,29.847629,129.416121
4,2.404168,0.167662,21.694021,195.814155
